In [ ]:

import pandas as pd

In [ ]:
trip = pd.read_csv(r'/content/drive/MyDrive/hotel_reviews.csv')

In [ ]:
trip.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [ ]:
trip = trip[(trip['Rating']==5)|(trip['Rating']==2)|(trip['Rating']==1)][['Review','Rating']]

In [ ]:
# lets modify the rating column
trip['Rating'] = trip['Rating'].apply(lambda rating: 'pos' if rating==5 else 'neg')

In [ ]:
#reseting the index
trip.reset_index(inplace=True)
trip.head()

,index,Review,Rating
0,1,ok nothing special charge diamond member hilto...,neg
1,3,"unique, great stay, wonderful time hotel monac...",pos
2,4,"great stay great stay, went seahawk game aweso...",pos
3,5,love monaco staff husband stayed hotel crazy w...,pos
4,6,"cozy stay rainy city, husband spent 7 nights m...",pos


In [ ]:
trip['Rating'].value_counts()

pos    9054
neg    3214
Name: Rating, dtype: int64

In [ ]:
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [ ]:
#lemmatization object
ps = WordNetLemmatizer()
corpus = []

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
for i in range (0,len(trip)):
  review = re.sub('[^a-zA-Z]'," ",trip['Review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.lemmatize(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join (review)
  corpus.append(review)
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


#trip .to_csv('tdata.csv',index=flase)


In [ ]:
from sklearn .feature_extraction .text import CountVectorizer
CV = CountVectorizer()

In [ ]:
X = CV.fit_transform(corpus).toarray()

In [ ]:
X.shape

(12268, 34569)

In [ ]:
y = pd.get_dummies(trip['Rating'])
y = y.iloc[:,1].values

In [ ]:
y

array([0, 1, 1, ..., 0, 0, 0], dtype=uint8)

In [ ]:
#train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=3)

In [ ]:
#Training model using naive bayes
from sklearn.naive_bayes import MultinomialNB
model1 = MultinomialNB().fit(X_train,y_train)
#for the accuracy
model1.score(X_test,y_test)

y_pred = model1.predict(X_test)

In [ ]:
#compare y test and y_pred
#confusion matrix is a 2*2 matrix and it  tells,
#how many number of elements are correctly predicted.

from sklearn.metrics import confusion_matrix
confusion_m = confusion_matrix(y_test,y_pred)

In [ ]:
confusion_m

array([[ 574,   69],
       [  44, 1767]])

In [ ]:
#checking accuarcy score

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)

In [ ]:
#print accuracy
accuracy

0.9539527302363489

In [ ]:
from sklearn.metrics import precision_score
precision_score(y_test,y_pred)

0.9624183006535948

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test,y_pred)

0.9757040309221424

In [ ]:
from sklearn.metrics import fbeta_score
fbeta_score(y_test,y_pred,beta=1)


0.9690156292843433

In [43]:
#checking the training model with custom input data
message = " the hotel was nice and comfy "

data = [message]
vect = CV.transform(data).toarray()
my_prediction = model1.predict(vect)
my_prediction_prob = model1.predict_proba(vect)

if  my_prediction==1:
  print("Positive")
  print(my_prediction_prob[0][1])
else:
     print("Negative")
     print(my_prediction_prob[0][0])

Positive
0.9689465878406804


In [ ]:
#checking the traning model with custom input data
message =" the hotel was bad and the staff was rude"
data = [message]
vect = CV.transform(data).toarray()
my_prediction = model1.predict(vect)
my_prediction_prob = model1.predict_proba(vect)

if my_prediction==1:
  print("Positive")
  print(my_prediction_prob[0][1])
else:
  print("Negative")
  print(my_prediction_prob[0][0])

Negative
0.8445454685022524


In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='liblinear')

In [ ]:
model = logreg.fit(X_train,y_train)

In [ ]:
y_pred_class = logreg.predict(X_test)

In [ ]:
from sklearn import metrics

In [ ]:
metrics.accuracy_score(y_test,y_pred)

0.9539527302363489

In [ ]:
metrics.roc_auc_score(y_test,y_pred)

0.9341972720707136

In [ ]:
metrics.confusion_matrix(y_test,y_pred)

array([[ 574,   69],
       [  44, 1767]])

In [ ]:
#checking the traning model with custom input data
message = "cozy stay rainy city,wonderful time at the hotel."
data = [message.lower()]
vect = CV.transform(data).toarray()
my_prediction = model.predict(vect)

if my_prediction==1:
  print("Positive")
  print(my_prediction_prob[0][1])
else:
  print("Negative")
  print(my_prediction_prob[0][0])

Positive
0.15545453149774663


In [ ]:

#checking the training model with custom input data
message ="the hotel was bad and the staff was rude"
data =[message.lower()]
vect = CV.transform(data).toarray()
my_prediction = model.predict(vect)
my_prediction_prob = model.predict_proba(vect)

if my_prediction==1:
  print("Positive")
  print(my_prediction_prob[0][1])
else:
  print("Negative")
  print(my_prediction_prob[0][0])

Negative
0.899664533424772


In [ ]:
#pickle file for logistic regression
import pickle

filename = 'logistic_regression_model.pkl'
pickle.dump(model,open(filename, 'wb'))
pickle.dump(CV,open('transform_logistic.pkl','wb'))

In [ ]:
#pickle file for naive bayes

pickle.dump(CV,open('transform.pkl','wb'))
filename = 'naive_bayes_model.pkl'
pickle.dump(model1,open(filename,'wb'))